# Data Cleaning

In this course, we will walk through the full data science life cycle, from data cleaning and feature selection to machine learning. We will focus on credit modelling, a well known data science problem that focuses on modeling a borrower's credit risk. Credit has played a key role in the economy for centuries and some form of credit has existed since the beginning of commerce. We'll be working with financial lending data from Lending Club. Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. You can read more about their marketplace here.

Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more. Lending Club evaluates each borrower's credit score using past historical data (and their own data science process!) and assign an interest rate to the borrower. The interest rate is the percent in addition to the requested loan amount the borrower has to pay back. You can read more about the interest rate that Lending Club assigns here. Lending Club also tries to verify each piece of information the borrower provides but it can't always verify all of the information (usually for regulation reasons).

A higher interest rate means that the borrower is riskier and more unlikely to pay back the loan while a lower interest rate means that the borrower has a good credit history is more likely to pay back the loan. The interest rates range from 5.32% all the way to 30.99% and each borrower is given a grade according to the interest rate they were assigned. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.

Investors are primarily interested in receiveing a return on their investments. Approved loans are listed on the Lending Club website, where qualified investors can browse recently approved loans, the borrower's credit score, the purpose for the loan, and other information from the application. Once they're ready to back a loan, they select the amount of money they want to fund. Once a loan's requested amount is fully funded, the borrower receives the money they requested minus the origination fee that Lending Club charges.

The borrower then makes monthly payments back to Lending Club either over 36 months or over 60 months. Lending Club redistributes these payments to the investors. This means that investors don't have to wait until the full amount is paid off to start to see money back. If a loan is fully paid off on time, the investors make a return which corresponds to the interest rate the borrower had to pay in addition the requested amount. Many loans aren't completely paid off on time, however, and some borrowers default on the loan.

Here's a diagram from Bible Money Matters that sums up the process:

<img src='loans/fig1.jpg'>

While Lending Club has to be extremely savvy and rigorous with their credit modelling, investors on Lending Club need to be equally as savvy about determining which loans are more likely to be paid off. While at first, you may wonder why investors would put money into anything but low interest loans. The incentive investors have to back higher interest loans is, well, the higher interest! If investors believe the borrower can pay back the loan, even if he or she has a weak financial history, then investors can make more money through the larger additional amount the borrower has to pay.

Most investors use a portfolio strategy to invest small amounts in many loans, with healthy mixes of low, medium, and interest loans. In this course, we'll focus on the mindset of a conservative investor who only wants to invest in the loans that have a good chance of being paid off on time. To do that, we'll need to first understand the features in the dataset and then experiment with building machine learning models that reliably predict if a loan will be paid off or not.

Lending Club releases data for all of the approved and declined loan applications periodically on their website. You can select a few different year ranges to download the datasets (in CSV format) for both approved and declined loans.

You'll also find a data dictionary (in XLS format) which contains information on the different column names towards the bottom of the page. We recommend downloading the data dictionary to so you can refer to it whenever you want to learn more about what a column represents in the datasets. Here's a link to the data dictionary file hosted on Google Drive.

Before diving into the datasets themselves, let's get familiar with the data dictionary. The LoanStats sheet describes the approved loans datasets and the RejectStats describes the rejected loans datasets. Since rejected applications don't appear on the Lending Club marketplace and aren't available for investment, we'll be focusing on data on approved loans only.

The approved loans datasets contain information on current loans, completed loans, and defaulted loans. Let's now define the problem statement for this machine learning project:

- Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

Before we can start doing machine learning, we need to define what features we want to use and which column repesents the target column we want to predict. Let's start by reading in the dataset and exploring it.

In this mission, we'll focus on approved loans data from 2007 to 2011, since a good number of the loans have already finished. In the datasets for later years, many of the loans are current and still being paid off.

To ensure that code runs fast on our platform, we reduced the size of LoanStats3a.csv by:

- removing the first line:
     -because it contains the extraneous text Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action) instead of the column titles, which prevents the dataset from being parsed by the pandas library properly
- removing the desc column:
    - which contains a long text explanation for each loan
- removing the url column:
    - which contains a link to each loan on Lending Club which can only be accessed with an investor account
- removing all columns containing more than 50% missing values:
    - which allows us to move faster since we can spend less time trying to fill these values

The following code replicates this process, if you want to replicate the dataset to work with it on your own:

    import pandas as pd
    loans_2007 = pd.read_csv('LoanStats3a.csv', skiprows=1)
    half_count = len(loans_2007) / 2
    loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)
    loans_2007 = loans_2007.drop(['desc', 'url'],axis=1)
    loans_2007.to_csv('loans_2007.csv', index=False)

We named the filtered dataset loans_2007.csv instead in case we want to explore the raw dataset (LoanStats3a.csv) without mixing up the two. First things first, let's read in the dataset into a Dataframe so we can start to explore the data and explore the remaining features.

<b>Instructions</b><br>
Read loans_2007.csv into a DataFrame named loans_2007 and use the print function to display the first row of the Dataframe.

Use the print function to:

- display the first row of loans_2007 and
- the number of columns in loans_2007.

In [1]:
import pandas as pd

In [8]:
loans = pd.read_csv('loans_2007.csv', low_memory=False)
loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


The Dataframe contains many columns and can be cumbersome to try to explore all at once. Let's break up the columns into 3 groups of 18 columns and use the data dictionary to become familiar with what each column represents. As you understand each feature, you want to pay attention to any features that:

- leak information from the future (after the loan has already been funded)
- don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
- formatted poorly and need to be cleaned up
- require more data or a lot of processing to turn into a useful feature
contain redundant information

We need to especially pay attention to data leakage, since it can cause our model to overfit. This is because the model would be using data about the target column that wouldn't be available when we're using the model on future loans. We encourage you to spend as much time as you need to understand each column, because a poor understanding could cause you to make mistakes in the data analysis and modeling process. As you go through the dictionary, keep in mind that we need to select one of the columns as the target column we want to use when we move on to the machine learning phase.

In this screen and the next few screens, let's focus on just columns that we need to remove from consideration. Then, we can circle back and further dissect the columns we decided to keep.

To make this process easier, we created a table that contains the name, data type, first row's value, and description from the data dictionary for the first 18 rows.

<img src='loans/fig2.jpg'>

After analyzing each column, we can conclude that the following features need to be removed:

- id: randomly generated field by Lending Club for unique identification purposes only
- member_id: also a randomly generated field by Lending Club for unique identification purposes only
- funded_amnt: leaks data from the future (after the loan is already started to be funded)
- funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
- grade: contains redundant information as the interest rate column (int_rate)
- sub_grade: also contains redundant information as the interest rate column (int_rate)
- emp_title: requires other data and a lot of processing to potentially be useful
- issue_d: leaks data from the future (after the loan is already completed funded)

Recall that Lending Club assigns a grade and a sub-grade based on the borrower's interest rate. While the grade and sub_grade values are categorical, the int_rate column contains continuous values, which are better suited for machine learning.

Let's now drop these columns from the Dataframe before moving onto the next group of columns.

In [10]:
cols_to_drop = ['id', 'member_id', 'funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade', 'emp_title', 'issue_d']

loans.drop(cols_to_drop, axis=1, inplace=True)

Let's now look at the next 18 columns:

<img src='loans/fig3.jpg'>

Within this group of columns, we need to drop the following columns:

- zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
- out_prncp: leaks data from the future, (after the loan already started to be paid off)
- out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)

The out_prncp and out_prncp_inv both describe the outstanding principal amount for a loan, which is the remaining amount the borrower still owes. These 2 columns as well as the total_pymnt column describe properties of the loan after it's fully funded and started to be paid off. This information isn't available to an investor before the loan is fully funded and we don't want to include it in our model.

Let's go ahead and remove these columns from the Dataframe.

In [12]:
cols_to_drop = ['zip_code', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp']
loans.drop(cols_to_drop, axis=1, inplace=True)

Let's now move on to the last group of features:

<img src='loans/fig4.jpg'>

In the last group of columns, we need to drop the following columns:

- total_rec_int: leaks data from the future, (after the loan already started to be paid off),
- total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
- recoveries: also leaks data from the future, (after the loan already started to be paid off),
- collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
- last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
- last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).

All of these columns leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower.

In [13]:
cols_to_drop = ['total_rec_int', 'total_rec_late_fee', 'recoveries', 
                'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt']

loans.drop(cols_to_drop, axis=1, inplace=True)

Just by becoming familiar with the columns in the dataset, we were able to reduce the number of columns from 52 to 32 columns. We now need to decide on a target column that we want to use for modeling.

We should use the loan_status column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one for training a model. Let's explore the different values in this column and come up with a strategy for converting the values in this column.

<b>Instructions</b><br>
Use the Dataframe method value_counts to return the frequency of the unique values in the loan_status column.
Display the frequency of each unique value using the print function.

In [14]:
loans['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

There are 8 different possible values for the loan_status column. You can read about most of the different loan statuses on the <a href='http://kb.lendingclub.com/investor/articles/Investor/What-do-the-different-Note-statuses-mean/'>Lending Club website</a>. The 2 values that start with "Does not meet the credit policy" aren't explained unfortunately. A quick Google search takes us to explanations from the lending comunity <a href='http://www.lendacademy.com/forum/index.php?topic=2427.msg20813#msg20813'>here</a> and <a href='http://www.orchardplatform.com/blog/2014520loans-issued-under-previous-credit-policies/'>here</a>.

We've compiled the explanation for each column as well as the counts in the Dataframe in the following table:

<img src='loans/fig5.jpg'>
          
From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the Fully Paid and Charged Off values describe the final outcome of the loan. The other values describe loans that are still on going and where the jury is still out on if the borrower will pay back the loan on time or not. While the Default status resembles the Charged Off status, in Lending Club's eyes, loans that are charged off have essentially no chance of being repaid while default ones have a small chance. You can read about the difference <a href='http://kb.lendingclub.com/investor/articles/Investor/What-is-the-difference-between-a-loan-that-is-in-default-and-a-loan-that-has-been-charged-off'>here</a>.

Since we're interested in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a <b>binary classification</b> one. Let's remove all the loans that don't contain either Fully Paid and Charged Off as the loan's status and then transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case. While there are a few different ways to transform all of the values in a column, we'll use the <a href='http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.replace.html'>Dataframe method replace</a>. According to the documentation, we can pass the replace method a nested mapping dictionary in the following format:

    mapping_dict = {
        "date": {
            "january": 1,
            "february": 2,
            "march": 3
        }
    }
    df = df.replace(mapping_dict)
    
Lastly, one thing we need to keep in mind is the <b>class imbalance</b> between the positive and negative cases. While there are 33,136 loans that have been fully paid off, there are only 5,634 that were charged off. This class imbalance is a common problem in binary classification and during training, the model ends up having a strong bias towards predicting the class with more observations in the training set and will rarely predict the class with less observations. The stronger the imbalance, the more biased the model becomes. There are a few different ways to tackle this class imbalance, which we'll explore later.

In [16]:
mapping_dict = {
    'loan_status': {
        'Fully Paid': 1,
        'Charged Off': 0
    }
}

loans = loans.replace(mapping_dict)

In [17]:
loans['loan_status'].value_counts()

1                                                      33136
0                                                       5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

In [20]:
loans = loans[(loans['loan_status'] == 0) | (loans['loan_status'] == 1)]

In [21]:
loans['loan_status'].value_counts()

1    33136
0     5634
Name: loan_status, dtype: int64

To wrap up this mission, let's look for any columns that contain only one unique value and remove them. These columns won't be useful for the model since they don't add any information to each loan application. In addition, removing these columns will reduce the number of columns we'll need to explore further in the next mission.

We'll need to compute the number of unique values in each column and drop the columns that contain only one unique value. While the Series method unique returns the unique values in a column, it also counts the Pandas missing value object nan as a value:

    # Returns 0 and nan.
    unique_values = loans['tax_liens'].unique()

Since we're trying to find columns that contain one true unique value, we should first drop the null values then compute the number of unique values:

    non_null = loans_2007['tax_liens'].dropna()
    unique_non_null = non_null.unique()
    num_true_unique = len(unique_non_null)
    
<b>Instructions</b>
- Remove any columns from loans_2007 that contain only one unique value:
    - Create an empty list, drop_columns to keep track of which columns you want to drop
- For each column:
    - Use the Series method dropna to remove any null values and then use the Series method unique to return the set of non-null unique values
    - Use the len() function to return the number of values in that set
    - Append the column to drop_columns if it contains only 1 unique value
- Use the Dataframe method drop to remove the columns in drop_columns from loans_2007
- Use the print function to display drop_columns so we know which ones were removed

In [22]:
loans.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'pymnt_plan', 'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'initial_list_status',
       'last_credit_pull_d', 'collections_12_mths_ex_med', 'policy_code',
       'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths',
       'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

In [23]:
def find_single_valued(df):
    """Returns the names of columns in df with single values (not including np.nan)"""
    
    single_valued = []
    for col in df.columns:
        if len(df[col].dropna().unique()) == 1:
            single_valued.append(col)
    return single_valued

In [25]:
drop_cols = find_single_valued(loans)
loans.drop(drop_cols, axis=1, inplace=True)

In [26]:
drop_cols

['pymnt_plan',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'tax_liens']

It looks we we were able to remove 9 more columns since they only contained 1 unique value.

In this mission, we started to become familiar with the columns in the dataset and removed many columns that aren't useful for modeling. We also selected our target column and decided to focus our modeling efforts on binary classification. In the next mission, we'll explore the individual features in greater depth and work towards training our first machine learning model.

# Features